In [2]:
"""
Author: Brent Glassman
1. Queries frequence_network_details for updated domains
2. Check to see if domains are valid
3. Pull Network bidlists and check to see if they need to be updated based on if there is any difference between the new sites and the bidlists
4. Update Networks' bidlists that have new sites 

"""
import pandas as pd
import time
import sys
import numpy as np
sys.path.append(r"../../Desktop/performance_team_bidlist_code/")

import APIs.TTD_API_Library as tal
import APIs.Brandcdn_Library as blib

import json
import requests
# import os
# from pandas.io.json import json_normalize
import re
import datetime
cur_day = datetime.datetime.utcnow().strftime("%m/%d/%y")

In [3]:
prod_query = """
select fnd.network_name as network_name
,nm.domain as domain
,nm.frequence_network_id as network_id
FROM brandcdn.frequence_network_details fnd
JOIN brandcdn.ctv_site_to_network_and_aggregator_mapping nm
ON nm.frequence_network_id=fnd.id
ORDER BY 1
"""
bidlists_query = """
select *
FROM frequence_network_join_bidlist
"""

In [4]:
df2 = blib.bcdn_select_query(prod_query, connection = blib.bcdn_connection())
df3 = blib.bcdn_select_query(bidlists_query, connection = blib.bcdn_connection())

In [5]:
df3 = df3[df3.columns[1:]] # Get rid of some random Ids column

In [6]:
df_use = df3.merge(df2, how = 'left', on = 'network_id')
df_use = df_use[df_use['domain'].notna()]
df_use

,network_id,bidlist_id,bidlist_name,bidlist_type,created_timestamp,updated_timestamp,network_name,domain
0,1250,65701680,Network BlockList WDBJ News Test 2020/11/17,BlockList,2020-12-11 00:03:59,2020-12-11 00:03:59,WDBJ News,com.roku.wdbjnews
1,1250,65701680,Network BlockList WDBJ News Test 2020/11/17,BlockList,2020-12-11 00:03:59,2020-12-11 00:03:59,WDBJ News,com.graytv.wdbjnews
2,175,65701681,Network BlockList History Test 2020/11/17,BlockList,2020-12-11 00:03:59,2020-12-11 00:03:59,History,com.sling.otto_roku_history_live
3,175,65701681,Network BlockList History Test 2020/11/17,BlockList,2020-12-11 00:03:59,2020-12-11 00:03:59,History,com.sling.otto_android_tv_history_live
4,175,65701681,Network BlockList History Test 2020/11/17,BlockList,2020-12-11 00:03:59,2020-12-11 00:03:59,History,com.history.chromecast.app
...,...,...,...,...,...,...,...,...
24787,720,71183300,"Network - 10 News, Type - TargetList, 2020/12/18",TargetList,2021-02-03 00:55:15,2021-02-03 00:55:15,10 News,com.10news
24790,862,71183365,"Network - American Heroes Channel, Type - Targ...",TargetList,2021-03-25 18:22:18,2021-03-25 18:22:18,American Heroes,americanheroes.com
24791,862,71183366,"Network - American Heroes Channel, Type - Bloc...",BlockList,2021-03-25 18:22:18,2021-03-25 18:22:18,American Heroes,americanheroes.com
24814,1064,71186286,"Network - Nickelodeon, Type - BlockList, 2020/...",BlockList,2021-03-26 01:20:07,2021-03-26 01:20:07,Nick Jr.,firetv.fubotv.nick jr.


In [7]:
supply_vendors = pd.read_csv('../assemblingbidlistsandtesting/SupplyVendorNames.csv')
adformats = pd.read_csv('../assemblingbidlistsandtesting/AdFormat.csv')
dimension_translation = {'Supply_Vendor': 'SupplyVendorId',  # Pull from a csv
                        'Fold': 'PlacementPositionRelativeToFold',  # Use the translation below
                        'Browser': 'Browser',  # Use the translation below
                        'Operating_System': 'Os',   # Use the translation below
                        'site': "DomainFragment",  # Use the default cut-off-validation
                        'adformat': 'AdFormatId',  # Pull from a csv
                         'renderingcontext': 'RenderingContext',
                        'devicename': 'DeviceType',  # Default
                         'supplyvendorpublisherid': "PublisherId"
                        }

bidline_translation = {'Fold': {'Above': ['Above'],
                                'Below': ['Below'], 
                                'Unknown': ['Unknown'], 
                                'Other': ['Any']
                               },
                       'DeviceType': {'Mobile': ['Mobile'],
                                'Tablet': ['Tablet'], 
                                'PC': ['Pc']
                               },
                        'Browser': {'Internet_Explorer_Old': ['InternetExplorer7', 'InternetExplorer8', 'InternetExplorer9'],
                                    'FireFox': ['Firefox'],
                                    'Chrome': ['Chrome'],
                                    'Safari': ['Safari'],
                                    'Opera': ['Opera'],
                                    'Internet_Explorer_New': ['InternetExplorer10', 'InternetExplorer11'],
                                    'Edge': ['Edge'],
                                    'Baidu': ['Baidu'],
                                    'Yandex': ['Yandex'],
                                    'WebView': ['WebView'],
                                     'Other': ['Other' , 'InApp']
                                   },
                        'Operating_System': {'Windows': ['WindowsAll','WindowsXP','WindowsVista','Windows7','Windows8','Windows10'],
                                            'OS X': ['OSXAll'],
                                            'Linux': ['LinuxAll'],
                                            'iOS': ['iOS51','iOS60','iOS61','iOS70','iOS71','iOS80','iOS81','iOS82',
                                                    'iOS83','iOS84','iOS90','iOS91','iOS92','iOS93','iOS100','iOS101',
                                                    'iOS102','iOS103','iOS110','iOS111','iOS112','iOS113','iOS120','iOS121',
                                                    'iOS122','iOS123','iOS124','iOS130','iOS131','iOS132','iOS133','iOS134',
                                                    'iOS140','iOS141','iOS142','iOS143','iOS135', 'iOSAll'],
                                            'Android': ['Android23','Android40','Android41','Android42','Android43',
                                                        'Android44','Android50','Android60','Android70', 'Android71',
                                                        'Android51','Android80','Android81','Android90','Android100',
                                                        'Android110','AndroidAll'],
                                            'Windows Phone': ['WindowsPhone80','WindowsPhone81', 'WindowsPhoneAll'],
                                            'Other': ['Other', 'OtherAll']}
                        }
def check_valid(domain):
    if_valid = True
    
    # getting rid of the part after the first '/'
    domain = str(domain).split('/')[0]
    
    # and check if the remaining contains any non-alphanumeric characters besides '.' and '-'
    if not domain.replace('.', '').replace('-', '').replace('_','').isalnum():
        if_valid = False
    elif domain[0] in ('.', '-','_') or domain[-1] in ('.', '-','_'):
        # do not allow '.' or '-' at the beginning or ending
        if_valid = False
    
    return if_valid

def make_payload(li, qa = None, bidlist_type = 'BlockList',Network = None):
    
    if bidlist_type == 'BlockList':
        bid_adjustment = 0
    elif bidlist_type == 'TargetList':
        bid_adjustment = 1
    
    payload_list = []
    #for i in range(len(li)):
    for i in range(1):
        #df = li[i]
        df = li
        d1 = df.columns[0]
        dt1 = dimension_translation[d1] # Get the dimension name
        
        if qa:
            bidlist_name = 'High CTR Global {}_{}_{} QAIV60 Rank{} {} 2020/{}'.format(d1, i+1, bidlist_type, cur_day)
        if not qa:
            #bidlist_name = 'High CTR Global {}_{}_{} Rank{} {} 2020/{}'.format(d1, d2, d3, i+1, bidlist_type, cur_day)
            bidlist_name = 'Network - {}, Type - {}, 2020/{}'.format(Network,bidlist_type, cur_day)

            
        partner_id = 'nc31odz'
        bidlines = []
        for i in range(len(df)):
            # Get translation for dimensions
            c1 = df.loc[i, d1]
            #c2 = df.loc[i, d2]
            #c3 = df.loc[i, d3]
            
            if d1 == "site":
                if check_valid(c1.split('/')[0]):
                    content1 = [c1.split('/')[0]]
                else:
                    content1 = []
                
            for i1 in content1:
                if qa:
                    bidline = {"BidAdjustment": bid_adjustment,
                               "DisplayViewabilityScoreRange": {
                                                                    "Min": qa,
                                                                    "Max": 1
                                                                },
                               dt1: i1}
                               #,dt3: i3}
                else:
                    bidline = {"BidAdjustment": bid_adjustment,
                               dt1: i1}
                               #,dt3: i3}
                if bidline not in bidlines:
                    bidlines.append(bidline)

        payload = {"Name": bidlist_name,
              "BidListAdjustmentType": bidlist_type,
               "ResolutionType": "ApplyMultiplyAdjustment",
              "BidLines": bidlines,
              "BidListOwner": "Partner",
              "BidListOwnerId": partner_id,
              "IsAvailableForLibraryUse": True
            }
        payload_list.append(json.dumps(payload))
    return payload_list

In [8]:
df = df_use
print(len(df['network_name'].unique()), "Unique Networks")
df['valid'] = [check_valid(val) for val in df['domain']]
cond = df['valid'] # Only include valid domains


548 Unique Networks


In [9]:
df = df[cond]
df = df.rename(columns = {'domain':'site'})
df = df.reset_index(drop=True)
df

,network_id,bidlist_id,bidlist_name,bidlist_type,created_timestamp,updated_timestamp,network_name,site,valid
0,1250,65701680,Network BlockList WDBJ News Test 2020/11/17,BlockList,2020-12-11 00:03:59,2020-12-11 00:03:59,WDBJ News,com.roku.wdbjnews,True
1,1250,65701680,Network BlockList WDBJ News Test 2020/11/17,BlockList,2020-12-11 00:03:59,2020-12-11 00:03:59,WDBJ News,com.graytv.wdbjnews,True
2,175,65701681,Network BlockList History Test 2020/11/17,BlockList,2020-12-11 00:03:59,2020-12-11 00:03:59,History,com.sling.otto_roku_history_live,True
3,175,65701681,Network BlockList History Test 2020/11/17,BlockList,2020-12-11 00:03:59,2020-12-11 00:03:59,History,com.sling.otto_android_tv_history_live,True
4,175,65701681,Network BlockList History Test 2020/11/17,BlockList,2020-12-11 00:03:59,2020-12-11 00:03:59,History,com.history.chromecast.app,True
...,...,...,...,...,...,...,...,...,...
22953,557,71189267,"Network - Zoom, Type - BlockList, 2020/12/18",BlockList,2021-02-02 21:55:42,2021-02-02 21:55:42,Zoom,com.sling.otto_tvos_zoom_live,True
22954,720,71183300,"Network - 10 News, Type - TargetList, 2020/12/18",TargetList,2021-02-03 00:55:15,2021-02-03 00:55:15,10 News,roku.10news,True
22955,720,71183300,"Network - 10 News, Type - TargetList, 2020/12/18",TargetList,2021-02-03 00:55:15,2021-02-03 00:55:15,10 News,com.10news,True
22956,862,71183365,"Network - American Heroes Channel, Type - Targ...",TargetList,2021-03-25 18:22:18,2021-03-25 18:22:18,American Heroes,americanheroes.com,True


In [10]:
d_IdNetwork = dict(zip(df.bidlist_id, df.network_name)) # {Bidlistid : NetworkName} 

In [11]:
bidlistIds = df['bidlist_id'].unique() # Unique bidlists to check and update

In [12]:
auth_token = tal.authenticate(login = 'ttd_api_nc31odz@frequence.com', password = '6}YKJK0L=/dHyK')

Step 1: Acquire authentication token
Posting authentication request to https://api.thetradedesk.com/v3/authentication...
Got auth token: GeCkBuICHhDUGN3BKXLxuJkonYmMLmoTPATaNgxD+DS6RyadkfbsnomgNTE6wF4CjzT1e9uXyIs5sjI8FniQXA==


# Quick Warning, If you start seeing messages like this:
Program paused for 10 sec.
Program paused for 20 sec.

That means you're trying to update a bidlist that's invalid. Need to break out of look and figure out why the new bidlist is weird. 

In [24]:
# Check Each Bidlist ID For Updated Sitelist. Push new Payload of new
for i in range(len(bidlistIds)):
    if(i%20==0):
        print(i, ' / ', len(bidlistIds),' done')
    payload = tal.get_bidlist_payload(bidlistIds[i], auth_token) # Pull bidlist from TTD
    payload = payload[1] # Grabs payload part
    
    sites = list(df[df['bidlist_id'] == bidlistIds[i]]['site']) # Gets updated list of network sites from query
    bl_type = df[df['bidlist_id'] == bidlistIds[i]]['bidlist_type'].unique()[0] # Is this bidlist a blocklist or targetlist? returns 'Blocklist' or 'Targetlist'
    name_output = 'Network ' + bl_type + ' - ' + d_IdNetwork[bidlistIds[i]] + ' - ' + cur_day # Makes a new name with the update date for the network list
    
    new_bl = [{'BidAdjustment': 1.0 if bl_type=='TargetList' else 0.0, 'DomainFragment': x,'VolumeControlPriority': 'Neutral'} for x in sites] # NEW BIDLINES
    
    # Compare oldsites and newsites. If they're the same no need to update with a new payload
    oldsites = set([x['DomainFragment'] for x in payload['BidLines']])
    newsites = set(sites)
    if(len(newsites - oldsites)==0):
        continue
    
    # replace old bidlines and Name
    payload['BidLines'] = new_bl # NEW PAYLOAD
    payload['Name'] = name_output # NEW NAME
    
    new = json.dumps(payload) # NEW PAYLOAD IN JSON
    tal.update_ttd_bidlist(new, auth_token) # Push new payload to TTD

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100


# Just if you're curious, It doesn't appear that domains with spaces are valid. Believe me I've tried

# Check on individual bidlists here if there are any issues

In [13]:
i=764 # find the bidlist id you want to check using np.where(bidlistIds == 'your bidlist id here')
payload = tal.get_bidlist_payload(bidlistIds[i], auth_token) # Pull bidlist from TTD
payload = payload[1] # Grabs payload part
oldsites = set([x['DomainFragment'] for x in payload['BidLines']])
oldsites

{'35059',
 '576009463',
 'att.mobile.history',
 'att.ott.history',
 'b00e5nh46q',
 'com-roku-hist.app',
 'com.ae.history',
 'com.ae.history.appletv',
 'com.ae.history.chromecast',
 'com.ae.history.roku',
 'com.ae.history.samsung',
 'com.aetn-history-watch.app',
 'com.aetn.history.ios.watch',
 'com.aetn.history.watch',
 'com.aetn.ios.historyvault',
 'com.appletv.history',
 'com.chromecast-history.app',
 'com.fubotv.firetv.historychannel',
 'com.fubotv.roku.historychannel',
 'com.history.appletv.app',
 'com.history.channel.documentaries.history.videos',
 'com.history.chromecast.app',
 'com.history.firetv.app',
 'com.history.mvpd.ctv',
 'com.history.mvpd.ctvapp',
 'com.history.roku.app',
 'com.history.smarttv.app',
 'com.roku-history.app',
 'com.roku.history',
 'com.sling.otto_airtvplayer_history_live',
 'com.sling.otto_airtvplayer_history_vod',
 'com.sling.otto_amazon_tablet_history_live',
 'com.sling.otto_amazon_tablet_history_vod',
 'com.sling.otto_amazon_tv_history_live',
 'com.sling.

In [14]:
sites = list(df[df['bidlist_id'] == bidlistIds[i]]['site'])
bl_type = df[df['bidlist_id'] == bidlistIds[i]]['bidlist_type'].unique()[0] #blocklist or targetlist
name_output = 'Network ' + bl_type + ' - ' + d_IdNetwork[bidlistIds[i]] + ' - ' + cur_day

In [15]:
new_bl = [{'BidAdjustment': 1.0 if bl_type=='TargetList' else 0.0, 'DomainFragment': x,'VolumeControlPriority': 'Neutral'} for x in sites] # NEW BIDLINES

In [16]:
payload['BidLines'] = new_bl # NEW PAYLOAD
payload['Name'] = name_output

In [17]:
newsites = set(sites)
print(len(newsites - oldsites), 'new domains')

0 new domains


In [18]:
new = json.dumps(payload) # NEW PAYLOAD IN JSON
new

'{"IsGlobal": false, "BidListDimensions": ["HasDomainFragmentId"], "BidListType": "BidAdjustment", "BidListId": "71184824", "Name": "Network BlockList - History - 05/19/22", "BidListSource": "User", "BidListAdjustmentType": "BlockList", "ResolutionType": "ApplyMultiplyAdjustment", "BidLines": [{"BidAdjustment": 0.0, "DomainFragment": "com.sling.otto_roku_history_live", "VolumeControlPriority": "Neutral"}, {"BidAdjustment": 0.0, "DomainFragment": "com.sling.otto_android_tv_history_live", "VolumeControlPriority": "Neutral"}, {"BidAdjustment": 0.0, "DomainFragment": "com.history.chromecast.app", "VolumeControlPriority": "Neutral"}, {"BidAdjustment": 0.0, "DomainFragment": "com.xumo.aenetworkthehistorychannel", "VolumeControlPriority": "Neutral"}, {"BidAdjustment": 0.0, "DomainFragment": "com.fubotv.roku.historychannel", "VolumeControlPriority": "Neutral"}, {"BidAdjustment": 0.0, "DomainFragment": "com.history.mvpd.ctvapp", "VolumeControlPriority": "Neutral"}, {"BidAdjustment": 0.0, "Domai

In [19]:
tal.update_ttd_bidlist(new, auth_token)

(True,
 {'IsGlobal': False,
  'BidListDimensions': ['HasDomainFragmentId'],
  'BidListType': 'BidAdjustment',
  'BidListId': '71184824',
  'Name': 'Network BlockList - History - 05/19/22',
  'BidListSource': 'User',
  'BidListAdjustmentType': 'BlockList',
  'ResolutionType': 'ApplyMultiplyAdjustment',
  'BidLines': [{'BidLineId': '4544470669',
    'BidAdjustment': 0.0,
    'DomainFragment': 'www.history.com',
    'VolumeControlPriority': 'Neutral'},
   {'BidLineId': '4544470670',
    'BidAdjustment': 0.0,
    'DomainFragment': 'com.slingtv.ctv.history.roku',
    'VolumeControlPriority': 'Neutral'},
   {'BidLineId': '4544470671',
    'BidAdjustment': 0.0,
    'DomainFragment': 'com.ae.history',
    'VolumeControlPriority': 'Neutral'},
   {'BidLineId': '4544470672',
    'BidAdjustment': 0.0,
    'DomainFragment': 'com.history.mvpd.ctvapp',
    'VolumeControlPriority': 'Neutral'},
   {'BidLineId': '4544470673',
    'BidAdjustment': 0.0,
    'DomainFragment': 'com.slingtv.ctv.history.andro